In [1]:
%pip install openai langchain python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
# Cell 2: Load Environment Variables
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

# Cell 3: Define Prompt Generation Logic
from langchain import OpenAI, LLMChain
from langchain.prompts import PromptTemplate

# Initialize OpenAI with API key
llm = OpenAI(api_key=openai_api_key)

def generate_prompts(user_input, scenarios):
    template = """
    User input: {user_input}
    Scenario: {scenario}
    Generate a prompt for the above scenario.
    """
    prompts = []
    for scenario in scenarios:
        prompt_template = PromptTemplate(template=template, input_variables=["user_input", "scenario"])
        chain = LLMChain(llm=llm, prompt=prompt_template)
        result = chain.run(user_input=user_input, scenario=scenario)
        prompts.append(result)
    return prompts

# Cell 4: Example Usage
user_input = "Generate a marketing slogan for an eco-friendly product"
scenarios = ["short and catchy", "emphasizes sustainability", "targeted at young adults"]
generated_prompts = generate_prompts(user_input, scenarios)
for prompt in generated_prompts:
    print(prompt)


/home/melaku/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



"Think green, choose clean - for an eco-friendly future!" 

"What catchy phrase can you come up with to promote a sustainable and environmentally-friendly product?"

"Create a catchy slogan that will appeal to young adults and inspire them to make a positive impact on the environment with our eco-friendly product!"
